In [6]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import nltk
from nltk.corpus import stopwords

from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

path = 'google_credentials.json' 
client = language.LanguageServiceClient.from_service_account_json(path)

nltk.download('stopwords')
df_top10 = pd.read_excel('top_10_words.xls')

def preprocess(text):
    text = text.lower()
    text = text.replace(r'[^\w\s]+', '')
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return tokens

def get_score(tokens):
    score = dict.fromkeys(df_top10.columns, 0)
    for j in df_top10.columns:
        for l, l_in in zip(list(df_top10[j]), list(df_top10.index)):
            if l in tokens:
                score[j] += 10-l_in
    category = (max(score, key=score.get))
    score = score[max(score, key=score.get)]
    return score, category

def google_sentiment(text):
    # Score is positive or negative sentiment
    # Magnitude is how much the text displays that
    document = language.types.Document(
            content=text,
            type=language.enums.Document.Type.PLAIN_TEXT)
    annotations = client.analyze_sentiment(document=document)
    score = annotations.document_sentiment.score
    magnitude = annotations.document_sentiment.magnitude
    if (score <= 0.0):
        ex_negative = True
    else:
        ex_negative = False
    return ex_negative

def packaged_results(id, timestamp, message, user_name, contact_details):  #user_id, etc just given as examples
    category_score, category = get_score(preprocess(message))
    extreme_negative = google_sentiment(message)
    return {'id':id, 'timestamp':timestamp,'user_name':user_name, 'message': message, 'category': category, 'category_score': category_score, 'extreme_negative': extreme_negative}

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chris\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
packaged_results('id','time','Wo muss ich den Kehricht rausbringen?', 'user_name', 'contact_details')

NameError: name 'stop_words' is not defined

In [42]:
df_top10

,Glasfaser,Mobilitat,Umzug,Netz,Storungen,Internet,Wasser,Strom,Diverses,Kehricht,Warme
0,glasfasernetz,chf,umzug,kosten,klären,router,wasser,ökostrom,kunden,kehricht,erdgas
1,erschliessung,erdgas,melden,kunden,tel,ewbtv,schweiz,stromprodukt,smart,tonne,biogas
2,service,erdgasfahrzeuge,bringt,warum,ausgefallen,geräte,trinkwasser,kunden,zähler,pro,beim
3,eigentümer,gibt,meist,müssen,ab,ewbinternet,liter,produziert,meter,ab,etwa
4,anschluss,ebikes,bürokratischen,netzanschluss,handelt,kontrollieren,rund,kundinnen,teil,müssen,pro
5,angeschlossen,fahrzeuge,erledigungen,schweiz,nachbarn,nutzen,mineralwässer,bestellen,jahresrechnung,deponiert,jahr
6,angebote,akku,onlineformulare,trägt,störung,kundendienst,prozent,anteil,kunde,holz,preise
7,geschäftskunden,schweiz,erleichtern,swissgrid,rufen,fritzbox,hahnenwasser,hergestellt,kundinnen,erzeugt,beziehen
8,angeboten,km,abmeldung,schweizer,geplante,gerät,qualität,wählen,hydrospeicher,kehrichtmenge,kunden
9,finden,servicestelle,neue,netznutzung,beziehen,fall,natürlichen,stromverbrauch,energieversorger,zugenommen,ca
